In [1]:
import warnings
warnings.filterwarnings('ignore')
import random
#importing the libraries
import numpy as np
import pandas as pd

path="D:/Workspace/Projects/BayesianLearning/Project1_Data/CSV/"
test=pd.DataFrame(columns=['Actual'])

In [2]:
for k in [1,2,5,7,8]:
    #Session Data Splitter
    df=pd.read_csv(path+'Node_50'+str(k)+'.csv')
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%d/%m/00%y %H:%M')
    df['timediff']=df['timestamp'].diff().apply(lambda x: x/np.timedelta64(1, 'm')).fillna(0).astype('int64')
    cutlist=[0]
    for index, row in df.iterrows():
        if abs(row['timediff'])>=120:
            cutlist.append(index)
    cutlist.append(df.shape[0])
    
    for i in range(1,len(cutlist)):
        session=df.iloc[cutlist[i-1]:cutlist[i],:4]
        session.to_csv(path+'Sessions/PredS_N50'+str(k)+'_'+str(i).zfill(2)+'.csv',index=False)
        session.to_csv(path+'Sessions/PredSO_N50'+str(k)+'_'+str(i).zfill(2)+'.csv',index=False)
    
    df.dropna(inplace = True)
    df.reset_index(drop=True, inplace=True)

    #Random Test Data Generation
    rlist=[]
    while len(rlist)<80:
        val = random.randint(2, df.shape[0]-3)
        if val not in rlist and len(list(set([*range(val-2,val+3)]) & set(rlist)))<=1:
            rlist.append(val)

    for index, row in df.iterrows():
        select = random.randint(0, 1)
        if index in rlist and select==0:
            new_row = pd.Series([row['temperature']], index=test.columns)
            test = test.append(new_row,ignore_index=True)
            df.at[index,'temperature'] = np.nan
        if index in rlist and select==1:
            new_row = pd.Series([row['humidity']], index=test.columns)
            test = test.append(new_row,ignore_index=True)
            df.at[index,'humidity'] = np.nan
    
    #Test Data Splitter
    df.to_csv(path+'Sessions/TN_50'+str(k)+'.csv',index=False)
    df['timediff']=df['timestamp'].diff().apply(lambda x: x/np.timedelta64(1, 'm')).fillna(0).astype('int64')
    testcutlist=[0]
    for index, row in df.iterrows():
        if abs(row['timediff'])>=120:
            testcutlist.append(index)
    testcutlist.append(df.shape[0])

    for i in range(1,len(testcutlist)):
        session=df.iloc[testcutlist[i-1]:testcutlist[i],:4]
        session.to_csv(path+'Sessions/TestS_N50'+str(k)+'_'+str(i).zfill(2)+'.csv',index=False)
        session.to_csv(path+'Sessions/TestSO_N50'+str(k)+'_'+str(i).zfill(2)+'.csv',index=False)

In [3]:
test = test.astype({'Actual':'float64'})
test.to_csv('D:/Workspace/Projects/BayesianLearning/Project1_Data/CSV/Sessions/TestActualData.csv',index=False)